# Preprocessing

In [ ]:
conda activate qiime2-2021.2

In [2]:
mkdir 2021_BNW_skin
mkdir 16SV4_all
cd 16SV4_all

In [ ]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \ 
  --input-path /Users/christinanaesborg/Desktop/Raw_Samples/16SV4_BNW/Collected \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux.qza

In [ ]:
qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

qiime tools view demux.qzv

In [ ]:
qiime metadata tabulate 
--m-input-file BNW-skin-stats-dada2.qza 
--o-visualization reports/BNW-skin-stats-dada2.qzv
qiime feature-table summarize 
--i-table BNW-skin-table.qza 
--o-visualization reports/BNW-skin-table.qzv
qiime feature-table tabulate-seqs 
--i-data BNW-skin-rep-seqs.qza 
--o-visualization reports/BNW-skin-rep-seqs.qzv

In [ ]:
# since sample id were only numbers, I had to reindex. Otherwise I would have incountered problems later on
qiime feature-table group \
  --i-table BNW-skin-table.qza \
  --p-axis sample \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column SampleID-new \
  --p-mode sum \
  --o-grouped-table BNW-skin-reindexed-table.qza

In [ ]:
qiime feature-table summarize \
  --i-table BNW-skin-reindexed-table.qza \
  --m-sample-metadata-file BNW-skin-metadata.tsv \
  --o-visualization BNW-skin-reindexed-table.qzv

# SILVA database

In [ ]:
# import Qiime2 artifacts
qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path /Users/christinanaesborg/Desktop/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna  \
  --output-path silva_99_otus.qza

In [ ]:
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path /Users/christinanaesborg/Desktop/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_all_levels.txt  \        
  --output-path silva_99_ref-taxonomy.qza

In [ ]:
# extract reference reads
qiime feature-classifier extract-reads \
  --i-sequences silva_99_otus.qza \
  --p-f-primer GTGCCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACHVGGGTWTCTAAT \
  --p-trunc-len 120 \
  --p-min-length 100 \
  --p-max-length 400 \
  --o-reads silva_ref-seqs.qza

In [ ]:
# train naive bayes classifier
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads silva_ref-seqs.qza \
  --i-reference-taxonomy silva_99_ref-taxonomy.qza \
  --o-classifier silva_132_99_classifier.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier silva_132_99_classifier.qza \
  --i-reads BNW-skin-rep-seqs.qza \
  --o-classification BNW-skin-silva-132.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-silva-132.qza \
--o-visualization BNW-skin-silva-132.qzv

In [ ]:
qiime fragment-insertion sepp \
--i-representative-sequences BNW-skin-rep-seqs.qza \
--i-reference-database sepp-refs-silva-128.qza \
--o-tree BNW-skin-sepp-tree.qza \
--o-placements BNW-skin-sepp-placements.qza \
--verbose \
--p-threads 2

In [ ]:
qiime fragment-insertion filter-features \
  --i-table BNW-skin-table.qza \
  --i-tree BNW-skin-sepp-tree.qza \
  --o-filtered-table BNW-skin-filtered_table.qza \
  --o-removed-table BNW-skin-removed_table.qza \
  --verbose

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-removed_table.qza \
--o-visualization BNW-skin-removed_table.qzv

Looks like nothing was removed, so will just use the unfiltered table (BNW-skin-table.qza) for subsequent analyses

In [ ]:
qiime diversity alpha-rarefaction \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file /Users/christinanaesborg/Desktop/Microbiome/BNW-skin-metadata.tsv   \
--o-visualization BNW-skin-reindexed-table-rarefaction.qzv \
--p-min-depth 500 \
--p-max-depth 50000

# output: BNW-skin-reindexed-table-rarefaction.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 12201 \
--o-filtered-table BNW-table-atleast-12201-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-table-atleast-12201-reads.qza \
--o-visualization BNW-table-atleast-12201-reads.qzv

# Taxa barplots

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads.qza \
--i-taxonomy BNW-skin-silva-132.qza \
--m-metadata-file BNW-skin-metadata.txt \
--o-visualization BNW-table-atleast-12201-reads-BARPLOT-PER-SAMPLE.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.txt \
--m-metadata-column infectionstatus \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-table-atleast-12201-reads-INFECTION-STATUS.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads-INFECTION-STATUS.qza \
--i-taxonomy BNW-skin-silva-132.qza \
--m-metadata-file BNW-skin-metadata-infection.txt \
--o-visualization BNW-table-atleast-12201-reads-INFECTION-STATUS-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.txt \
--m-metadata-column bodysite \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-table-atleast-12201-reads-BODY-SITE.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads-BODY-SITE.qza \
--i-taxonomy BNW-skin-silva-132.qza \
--m-metadata-file BNW-skin-metadata-BS.txt \
--o-visualization BNW-table-atleast-12201-reads-BODY-SITE-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.txt \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-table-atleast-12201-reads-WOMBAT.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads-WOMBAT.qza \
--i-taxonomy BNW-skin-silva-132.qza \
--m-metadata-file BNW-skin-metadata-WOM.txt \
--o-visualization BNW-table-atleast-12201-reads-WOMBAT-BARPLOT.qzv

# Alpha/Beta diversity

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences BNW-skin-rep-seqs.qza \
  --o-alignment silva-aligned-rep-seqs.qza \
  --o-masked-alignment silva-masked-aligned-rep-seqs.qza \
  --o-tree silva-unrooted-tree.qza \
  --o-rooted-tree silva-rooted-tree.qza

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny silva-rooted-tree.qza \
  --i-table BNW-skin-reindex-table.qza \
  --p-sampling-depth 12201 \
  --m-metadata-file /Users/christinanaesborg/Desktop/Microbiome/BNW-skin-metadata.tsv  \
  --output-dir silva-core-metrics

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity silva-core-metrics/faith_pd_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization silva-core-metrics/faith-pd-group-significance.qzv

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity silva-core-metrics/evenness_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization silva-core-metrics/evenness-group-significance.qzv

In [ ]:
# beta diversity - between body sites
qiime diversity beta-group-significance \
  --i-distance-matrix silva-core-metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column bodysite \
  --o-visualization silva-core-metrics/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - between wombats
qiime diversity beta-group-significance \
  --i-distance-matrix silva-core-metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column wombatid \
  --o-visualization silva-core-metrics/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - infection status
qiime diversity beta-group-significance \
  --i-distance-matrix silva-core-metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --m-metadata-column infectionstatus \
  --o-visualization silva-core-metrics/unweighted-unifrac-infection-status-significance.qzv \
  --p-pairwise

In [ ]:
qiime metadata tabulate \
  --m-input-file BNW-skin-metadata.tsv \
  --m-input-file faith_pd_vector.qza observed_features.qza\
  --o-visualization reindexed-tabulated-combined-metadata.qzv

# Export files to analyse in R

In [ ]:
# export unrooted tree
qiime tools export \
--input-path BNW-skin-sepp-tree.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
# export taxonomy
qiime tools export \
--input-path BNW-skin-silva-132.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
# export filtered table
qiime tools export \
--input-path BNW-table-atleast-12336-reads.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
qiime tools export \
--input-path observed_features.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
# Following script will change the first line of our file to desired header
sed 's/Feature ID/#OTUID/' /Users/christinanaesborg/Desktop/Microbiome/taxonomy.tsv | sed 's/Taxon/taxonomy/' | sed 's/Confidence/confidence/' > /Users/christinanaesborg/Desktop/Microbiome/biom-taxonomy.tsv

In [ ]:
biom add-metadata \
-i /Users/christinanaesborg/Desktop/Microbiome/feature-table.biom \
-o /Users/christinanaesborg/Desktop/Microbiome/table-with-taxonomyv2.biom \
--observation-metadata-fp /Users/christinanaesborg/Desktop/Microbiome/biom-taxonomy.tsv \
--sc-separated taxonomy

In [ ]:
qiime tools export 
--input-path faiths-pd-vector.qza 
--output-path /Users/christinanaesborg/Desktop/Microbiome/QIIME2_outputs/16S

# GreenGenes database - we ended up using SILVA in the final analysis

In [ ]:
qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path /Users/christinanaesborg/Desktop/gg_13_8_otus/rep_set/99_otus.fasta  \                            
  --output-path gg-99_otus.qza

In [ ]:
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path /Users/christinanaesborg/Desktop/gg_13_8_otus/taxonomy/99_otu_taxonomy.txt  \                             
  --output-path gg-99_ref-taxonomy.qz

In [ ]:
qiime vsearch cluster-features-closed-reference \
  --i-table BNW-skin-table.qza \
  --i-sequences BNW-skin-rep-seqs.qza \
  --i-reference-sequences gg-99_otus.qza \
  --p-perc-identity 0.99 \
  --o-clustered-table gg-99_cluster-table.qza \
  --o-unmatched-sequences unmatched.qza \
  --o-clustered-sequences gg-99_rep.qz

In [ ]:
# extract reference reads - primers are 515F/806R
qiime feature-classifier extract-reads \
  --i-sequences gg-99_otus.qza \
  --p-f-primer GTGCCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACHVGGGTWTCTAAT \
  --p-trunc-len 120 \
  --p-min-length 100 \
  --p-max-length 400 \
  --o-reads gg_ref-seqs.qza

In [ ]:
# train naive bayes classifier
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads gg_ref-seqs.qza \
  --i-reference-taxonomy gg-99_ref-taxonomy.qza \
  --o-classifier gg-99_classifier.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier gg-99_classifier.qza \
  --i-reads BNW-skin-rep-seqs.qza \
  --o-classification BNW-skin-gg_138.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-gg_138.qza \
--o-visualization BNW-skin-gg_138.qzv

In [ ]:
qiime fragment-insertion sepp \
--i-representative-sequences BNW-skin-rep-seqs.qza \
--i-reference-database /Users/christinanaesborg/Desktop/Microbiome_raw\ samples/sepp-refs-gg-13-8.qza  \              
--o-tree BNW-skin-sepp-tree-gg.qza \            
--o-placements BNW-skin-sepp-placements-gg.qza \
--verbose \  
--p-threads 2

In [ ]:
qiime fragment-insertion filter-features \
  --i-table BNW-skin-table.qza \
  --i-tree BNW-skin-sepp-tree-gg.qza \
  --o-filtered-table BNW-skin-filtered_table-gg.qza \
  --o-removed-table BNW-skin-removed_table-gg.qza \
  --verbose

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-removed_table-gg.qza \
--o-visualization BNW-skin-removed_table-gg.qzv

In [ ]:
# alpha rarefaction
qiime diversity alpha-rarefaction \
--i-table BNW-skin-table.qza \
--m-metadata-file /Users/christinanaesborg/Desktop/BNW-skin-metadata.txt   \
--o-visualization BNW-skin-table-rarefaction-gg.qzv \
--p-min-depth 500 \
--p-max-depth 50000

# output: BNW-skin-table-rarefaction.qzv

In [ ]:
# Generate phylogenetic tree for diversity analysis using mafft and fasttree
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences BNW-skin-rep-seqs.qza \
  --o-alignment gg-aligned-rep-seqs.qza \
  --o-masked-alignment gg-masked-aligned-rep-seqs.qza \
  --o-tree gg-unrooted-tree.qza \
  --o-rooted-tree gg-rooted-tree.qza

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny gg-rooted-tree.qza \
  --i-table BNW-skin-table.qza \
  --p-sampling-depth 12105 \
  --m-metadata-file /Users/christinanaesborg/Desktop/BNW-skin-metadata.tsv  \
  --output-dir gg-core-metrics-results

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity gg-core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --o-visualization gg-core-metrics-results/faith-pd-group-significance.qzv

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity gg-core-metrics-results/evenness_vector.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --o-visualization gg-core-metrics-results/evenness-group-significance.qzv

In [ ]:
# beta diversity - body site
qiime diversity beta-group-significance \
  --i-distance-matrix gg-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --m-metadata-column bodysite \
  --o-visualization gg-core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - individuals
qiime diversity beta-group-significance \
  --i-distance-matrix gg-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column wombatid \
  --o-visualization gg-core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - infection status
qiime diversity beta-group-significance \
  --i-distance-matrix gg-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --m-metadata-column infectionstatus \
  --o-visualization gg-core-metrics-results/unweighted-unifrac-infection-status-significance.qzv \
  --p-pairwise